### Import and Data preparation

In [ ]:
import yfinance as yf
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
# Define tickers for Mag 7 stocks and QQQ
tickers = ['AAPL', 'MSFT', 'AMZN', 'NVDA', 'GOOG', 'META', 'TSLA', 'QQQ']
data = yf.download(tickers, start='2022-01-01')[['Adj Close', 'Volume']]

In [ ]:
data

In [ ]:
# Separate price and volume data for the portfolio and benchmark
price_data = data['Adj Close']
volume_data = data['Volume']

In [ ]:
price_data

In [ ]:
volume_data

In [ ]:
# Get returns of QQQ
benchmark_returns = price_data.pop("QQQ").pct_change().dropna()[29:]

In [ ]:
benchmark_returns

In [ ]:
# Calculate 30-day rolling average volume for weighting
volume_data.pop("QQQ")
avg_volume = volume_data.rolling(window=30).mean()

In [ ]:
pd.set_option('display.float_format', '{:,.0f}'.format)
avg_volume

In [ ]:
# Calculate portfolio returns with volume-based weights
def calculate_weighted_returns(price_data, avg_volume):
    weights = avg_volume.div(avg_volume.sum(axis=1), axis=0)  # Normalize to get weights
    weighted_returns = (price_data.pct_change() * weights.shift(1)).sum(axis=1)
    return weighted_returns

In [ ]:
# Get weighted portfolio returns
portfolio_returns = calculate_weighted_returns(price_data, avg_volume)[30:]

In [ ]:
pd.reset_option('display.float_format')
portfolio_returns

In [ ]:
# Plot returns for visual check
plt.figure(figsize=(12, 8))
plt.plot(portfolio_returns[30:], label='Volume-Weighted Mag 7')
plt.plot(benchmark_returns[29:], label='QQQ')
plt.xlabel('Date')
plt.ylabel('Returns (in percentage)')
plt.title('QQQ vs Volume-Weighted Mag 7 Portfolio Returns')
plt.legend()
plt.show()

### Linear Regression and Hedge Ratio

In [ ]:
def linreg(x, y):    
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    return model

In [ ]:
X = benchmark_returns.values
Y = portfolio_returns.values

In [ ]:
print(len(X), len(Y))

In [ ]:
model = linreg(X, Y)
alpha, beta = model.params[0], model.params[1]

In [ ]:
print(model.summary())
print(f"Alpha: {alpha}")
print(f"Beta: {beta}")

### Implementing the Hedge

In [ ]:
hedged_portfolio_returns = -beta * benchmark_returns + portfolio_returns

P = hedged_portfolio_returns.values
model = linreg(X, P)
alpha, beta = model.params[0], model.params[1]

In [ ]:
print(model.summary())
print(f"Alpha: {alpha}")
print(f"Beta: {round(beta, 6)}")

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(hedged_portfolio_returns, label='Shorting QQQ, Longing Portfolio')
plt.plot(benchmark_returns, label='QQQ')
plt.xlabel('Date')
plt.ylabel('Returns (in percentage)')
plt.title('QQQ vs Hedged Portfolio')
plt.legend()

### Backtesting

In [ ]:
def hedged_portfolio_performance_dynamic_beta(
    portfolio_size,
    rebalancing_days,
    benchmark_returns,
    long_returns,
    short_returns,
    transaction_cost
):

    portfolio_value = portfolio_size
    portfolio_values = [portfolio_value]
    betas = []

    # Initial beta calculation for precise initial allocations
    initial_benchmark_returns = benchmark_returns[:rebalancing_days]
    initial_long_returns = long_returns[:rebalancing_days]

    # Calculate initial beta using OLS
    X = sm.add_constant(initial_benchmark_returns)
    model = sm.OLS(initial_long_returns, X).fit()
    initial_beta = model.params[1]
    betas.append(initial_beta)

    # Calculate initial long and short allocation ratios based on initial beta
    long_ratio = 1 / (1 + initial_beta)
    short_ratio = initial_beta / (1 + initial_beta)

    # Initial allocations
    long_allocation = portfolio_value * long_ratio
    short_allocation = portfolio_value * short_ratio

    # Loop through the returns, rebalancing every `rebalancing_days`
    for i in range(0, len(long_returns), rebalancing_days):
        # Get returns for the current rebalancing period
        long_period_returns = long_returns[i:i + rebalancing_days]
        short_period_returns = short_returns[i:i + rebalancing_days]
        benchmark_period_returns = benchmark_returns[i:i + rebalancing_days]

        # Calculate portfolio returns over the rebalancing period
        portfolio_period_returns = (long_period_returns + short_period_returns) / 2

        # Recalculate beta using OLS for each rebalancing period
        if len(benchmark_period_returns) == rebalancing_days:
            X = sm.add_constant(benchmark_period_returns)
            model = sm.OLS(portfolio_period_returns, X).fit()
            beta = model.params[1]
            betas.append(beta)

            # Update long and short allocation ratios based on new beta
            long_ratio = 1 / (1 + beta)
            short_ratio = beta / (1 + beta)
        else:
            # If there's insufficient data, continue with previous allocation ratios
            long_ratio = long_allocation / portfolio_value
            short_ratio = short_allocation / portfolio_value

        # Calculate cumulative returns over the rebalancing period
        long_cum_return = np.prod(1 + long_period_returns) - 1
        short_cum_return = np.prod(1 + short_period_returns) - 1

        # Update long and short allocations based on cumulative returns
        long_allocation *= (1 + long_cum_return) * (1 - transaction_cost)
        short_allocation *= (1 + short_cum_return) * (1 - transaction_cost)

        # Calculate total portfolio value after rebalancing
        portfolio_value = long_allocation + short_allocation
        portfolio_values.append(portfolio_value)

        # Rebalance the portfolio based on the new long/short ratio
        long_allocation = portfolio_value * long_ratio
        short_allocation = portfolio_value * short_ratio

    return portfolio_values, betas


In [ ]:
def get_sharpe_ratio(risk_free_rate, portfolio_values, rebalancing_days):
    # Calculate Sharpe Ratio
    portfolio_period_returns = pd.Series(portfolio_values).pct_change().dropna()  # Calculate returns for each rebalancing period
    period_risk_free_rate = (1 + risk_free_rate) ** (rebalancing_days / 365) - 1  # Adjusted to rebalancing frequency
    excess_returns = portfolio_period_returns - period_risk_free_rate

    # Annualize the Sharpe Ratio
    periods_per_year = 365 / rebalancing_days
    mean_excess_return = np.mean(excess_returns)
    std_dev_excess_return = np.std(excess_returns)
    sharpe_ratio = (mean_excess_return / std_dev_excess_return) * np.sqrt(periods_per_year)

    return sharpe_ratio

In [ ]:
# Example parameters for the hedged strategy
portfolio_size = 1000000
rebalancing_days = 30
short_returns = -benchmark_returns  # Assume shorting the benchmark as a basic hedge
transaction_cost = 0.001  # 0.1% per trade
risk_free_rate = 0.02  # Annual risk-free rate (2%)

# Run the hedged strategy
portfolio_values, betas = hedged_portfolio_performance_dynamic_beta(
    portfolio_size,
    rebalancing_days,
    benchmark_returns.values,
    portfolio_returns.values,  # Volume-weighted long returns
    short_returns.values,
    transaction_cost
)

sharpe_ratio = get_sharpe_ratio(risk_free_rate, portfolio_values, rebalancing_days)

In [ ]:
# Create a DataFrame for better presentation
if len(portfolio_values) != len(betas):
    min_length = min(len(portfolio_values), len(betas))
    portfolio_values = portfolio_values[:min_length]
    betas = betas[:min_length]
    print("Warning: Length mismatch detected. Trimming data to match lengths.")

df = pd.DataFrame({
    "Rebalancing Step": range(1, len(portfolio_values)+1),
    "Portfolio Value": portfolio_values,
    "Beta Value": betas
})

# Calculate initial, final portfolio size, and total realized returns
initial_portfolio_size = portfolio_values[0]
final_portfolio_size = portfolio_values[-1]
total_realized_returns = (final_portfolio_size - initial_portfolio_size) / initial_portfolio_size * 100
annualized_returns = ((final_portfolio_size / initial_portfolio_size) ** (365 / (min(len(portfolio_values),len(betas)) * rebalancing_days)) - 1) * 100

# Benchmark comparison
benchmark_cumulative_return = (1 + benchmark_returns).prod() - 1
benchmark_annualized_return = ((1 + benchmark_cumulative_return) ** (365 / len(benchmark_returns)) - 1) * 100

# Output results in a cleaner format
print(f"Initial Portfolio Size: {initial_portfolio_size:,.2f}")
print(f"Final Portfolio Size: {final_portfolio_size:,.2f}")
print(f"Total Realized Returns: {total_realized_returns:.2f}%")
print(f"Annual Returns: {annualized_returns:.2f}%")
print(f"Benchmark Annualized Returns: {benchmark_annualized_return:.2f}%")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}\n")


# Display the DataFrame with formatting
pd.options.display.float_format = '{:,.2f}'.format
print("Portfolio and Beta values at each rebalancing:\n", df)
